In [1]:
from numba import jit
import rasterio
from pathlib import Path
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from joblib import Parallel, delayed

C:\Users\ArdoJ\AppData\Roaming\Python\Python311\site-packages\scipy\__init__.py:169: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
c:\Users\ArdoJ\AppData\Local\miniconda3\envs\h3panda_exp\Lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\ArdoJ\AppData\Local\Temp\ipykernel_13684\1775464287.py:5: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'


In [4]:
def h3_index(gdf):
    import h3pandas
    h3=gdf.h3.geo_to_h3_aggregate(12, operation='mean')
    return h3

def process_file(file):
    with rasterio.open(file) as src:
        array = src.read(1)  # Read the first band
        transform = src.transform
        crs = src.crs  # Capture the CRS directly from the source

    # Generate full grid of coordinates
    cols, rows = array.shape
    col_indices, row_indices = np.meshgrid(np.arange(cols), np.arange(rows), indexing='xy')
    x_coords, y_coords = rasterio.transform.xy(transform, row_indices.flatten(), col_indices.flatten(), offset='center')
    
    # Flatten the array data to match the flat coordinate arrays
    values_flat = array.flatten()
    points = np.column_stack((x_coords, y_coords, values_flat))

    # Create GeoDataFrame
    geometry = [Point(x, y) for x, y, _ in points]
    df = gpd.GeoDataFrame(points, columns=['X', 'Y', 'Value'], geometry=geometry)
    df=df.drop(columns=['X','Y']).set_crs(2193)
    df=df.to_crs(4326)

    # H3 
    h3 = h3_index(df)
    h3.to_parquet(f'{output_dir}/{input_dir}_{file.stem}.parquet')

In [5]:
%%time
input_dir = Path('continuous')
output_dir = Path('combined_h3')
r_files = list(sorted(input_dir.glob('*.asc')))

# Using parallel processing Continuous
Parallel(n_jobs=-1)(delayed(process_file)(file) for file in r_files)
print('Done continuous!')

Done continuous!
CPU times: total: 2.28 s
Wall time: 1min 10s


In [6]:
%%time
input_dir = Path('discrete')
r_files = list(sorted(input_dir.glob('*.asc')))

# Using parallel processing Discrete
Parallel(n_jobs=-1)(delayed(process_file)(file) for file in r_files)
print('Done discrete!')

Done discrete!
CPU times: total: 2.3 s
Wall time: 1min 9s
